In [ ]:
import sys
sys.path.append("../../")

In [ ]:
from os.path import join, basename
from glob import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from typing import List, Any

from photometric.check_integrability import check_integrability
from photometric.estimate_alb_nrm import estimate_alb_nrm
from photometric.utils import convert_polar_to_cartesian

**Part 1 and 2**

In [ ]:
img_folder = "../SphereGray5/"
img_paths = glob(join(img_folder, "*.png"))

In [ ]:
# load images
imgs = [cv2.imread(x, 0) / 255.0 for x in img_paths]

# load ligh source directions (angles) from filenames
angs = [basename(x).split(".png")[0].split("_")[1:] for x in img_paths]
angs = np.array(angs).astype(float)

In [ ]:
img_stack = np.moveaxis(np.array(imgs), 0, -1)
scriptV = convert_polar_to_cartesian(angs[:, 0], angs[:, 1])

In [ ]:
albedo, normal = estimate_alb_nrm(img_stack, scriptV, shadow_trick=True)

In [ ]:
p, q, SE = check_integrability(normal)

In [ ]:
threshold = 0.005

In [ ]:
np.sum(SE < threshold) / np.sum(albedo > 0)

In [ ]:
num_outliers = np.sum(SE > threshold)

In [ ]:
albedo[np.where(SE < threshold)].max()

Testing with varying number of images

In [ ]:
img_folder = "../SphereGray25/"
img_paths = glob(join(img_folder, "*.png"))

# load images
imgs = [cv2.imread(x, 0) / 255.0 for x in img_paths]

# load ligh source directions (angles) from filenames
angs = [basename(x).split(".png")[0].split("_")[1:] for x in img_paths]
angs = np.array(angs).astype(float)

In [ ]:
num_imgs = [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25]
fraction_violating_test = []
num_outliers = []

for n in num_imgs:
    sub_imgs = imgs[:n]
    sub_angs = angs[:n]
    
    img_stack = np.moveaxis(np.array(sub_imgs), 0, -1)
    scriptV = convert_polar_to_cartesian(sub_angs[:, 0], sub_angs[:, 1])

    albedo, normal = estimate_alb_nrm(img_stack, scriptV, shadow_trick=True)
    
    p, q, SE = check_integrability(normal)
    
    num_outliers.append(np.sum(SE > threshold))
    fraction_violating_test.append(np.sum(SE < threshold) / np.sum(albedo > 0))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

ax.set_title("Integrability test with varying $n$")
ax.set_xlabel("Number of images $n$")
# ax.set_ylabel("Fraction of pixels where $SE >= \lambda (0.001)$")
ax.set_ylabel("Number of outliers")

# ax.plot(num_imgs, fraction_violating_test, 'o--')
ax.plot(num_imgs, num_outliers, 'o--')
ax.grid()

plt.show()